<img src="http://curiousily.com/assets/11.tensorflow_for_hackers_part_1_files/tensors_flowing.gif" width="50%" height="40%"/>

# Graphs

Everything you do in TensorFlow is based on computational graphs. Each node of the graph defines an operation. The operation can be any function - addition, multiplication or some funky custom function you wrote.

## Tensors

You can give a scalar value, vector, matrix or a Tensor to the operations in the graph. Tensors are just multi-dimensional matrices.

Let's have a look at an example computational graph:

<img src="https://cdn-images-1.medium.com/max/1600/1*mvhm5_r6LY-eHsin21RJTg.png" height="80%" width="80%"/>

How does this look like in TensorFlow?

In [15]:
import tensorflow as tf

t1 = tf.Variable(2)
t2 = tf.Variable(3)

res = t1 * t2

That was easy, right? No graphs, nothing seriously strange. And now for the result:

In [9]:
res

<tf.Tensor 'mul_3:0' shape=() dtype=int32>

That must be how new math is developed. $2 \times 3 = \text{tf.Tensor}$?

First, let's see the hidden `default` graph.

In [11]:
t1.graph is tf.get_default_graph()

True

In [12]:
res.graph is tf.get_default_graph()

True

All your Tensors are attached to the mystical `default` graph. What about the result?

# Sessions

See, the code above didn't perform any computation. You just created a computational graph. If you want to do something meaningful you need a `session`. Sessions are used for graph evaluation, think variable initialization and actually computing `res`.

A TensorFlow session takes care of running operations (on GPUs and CPUs) and holding variable values. Let's see how this is done:

In [18]:
with tf.Session() as sess:
    t1.initializer.run()
    t2.initializer.run()
    print(f"Multiplication result: {res.eval()}")

Multiplication result: 6


We initialized all our tensors and evaluated the result. Here is a somewhat shorter version that initializes all global variables automagically:

In [27]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run() # or sess.run(init)
    print(f"Multiplication result: {res.eval()}")

Multiplication result: 6


# Variables

Variables are just... eh, variables? By definition, you can change their values. That is quite useful, since tensors and operators are immutable. Let's create a normally distributed $3\times4$ matrix with $mean=0$ and $std=100$

In [31]:
normal = tf.Variable(tf.random_normal([3, 4], mean=0.0, stddev=100.0))

In [32]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(normal.eval())

[[  88.98575592  -63.97093964   22.90000916  -46.07382965]
 [ -93.86627197   21.56847191   71.74320221   49.92846298]
 [ -36.90287399   27.77186584   96.48136139 -177.17501831]]


Now let's change it:

In [34]:
normal = normal.assign(tf.Variable(tf.random_normal([3, 4], mean=0.0, stddev=100.0)))

In [35]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(normal.eval())

[[  39.33778763 -162.35063171 -122.39305878  168.09117126]
 [ -29.89160156  208.02810669 -179.69007874  -32.09128571]
 [-109.32617188  -30.70136261  162.00169373   66.11909485]]


Hard, right?